In [1]:
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir)

In [11]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle

In [4]:
# read input
dataset = os.path.join(currentdir, 'centralities.csv')
centralities = pd.read_csv(dataset, sep=';')
centralities.head()


,Author,Gender,MAC,PageRank,Katz,Degree
0,Agnes,1,1.0,5,5,6
1,Lasse,0,0.9,6,3,5
2,Kim,-1,1.0,1,1,1


In [5]:
# sort input, save top k only
def topK(k, centrality, data, removeUnknownGender):
    # check if centrality is valid?
    sortedCentrality = data.sort_values(by = centrality, ascending=False)
    if (removeUnknownGender):
        sortedCentrality = sortedCentrality.query('Gender != -1')
        
    renamed = sortedCentrality.rename(columns = {'Gender': 'item', centrality: 'rank'}, inplace=False)
    return renamed.head(k)

topK(k = 10, centrality='PageRank', data = centralities, removeUnknownGender=True)

,Author,item,MAC,rank,Katz,Degree
1,Lasse,0,0.9,6,3,5
0,Agnes,1,1.0,5,5,6


In [20]:
# evaluate rankings 
from lenskit.metrics.topnFair import *

protected=[0]

def equal_ex(k, centrality):
    return calculate_equal_ex(recs=topK(k, centrality, centralities, False), protected_group=protected)

def rnd(k):
    return 0

In [15]:
rla = topnFair.FairRecListAnalysis(['Author', "Centrality"])
rla.add_metric("rND")
rla.add_metric("rKL")
rla.add_metric("rRD")
rla.add_metric("div")
rla.add_metric("dem_parity")
rla.add_metric("ndcg")
results = rla.compute(all_recs_joined, test_data, "Degree", providers, items_n, proItems_n)
results.head()

NameError: name 'all_recs_joined' is not defined